# How to Search for Webpages on Google and Scrape Social Media URLS

### Objective: Use Python to search for an entity (e.g. a school or company) on Google. Navigate to the first Google search result and scrape any Twitter, Facebook, and Instagram pages.

The sample of schools I use for the college social media dataset comes from Brookings, a thinktank with extensive research about education and other public policy topics. In the data, the authors calculate "value added" for all major universities in the United States. Their value added measures represent how effective colleges are at increasing student earnings. (Learn more [here](https://www.brookings.edu/research/using-earnings-data-to-rank-colleges-a-value-added-approach-updated-with-college-scorecard-data/)) The data underlying the Brookings value added measures comes from the Integrated Postsecondary Education Data System ([IPEDS](https://nces.ed.gov/ipeds/)), which is maintained by the U.S. Department of education. 

## Set Up

#### Import Packages
The main packages we are using are: pandas, BeautifulSoup, and selenium. Pandas is used for data management. Selenium is used to implement the Google searches and get around Google's bot detection. BeautifulSoup and requests fetch data from the webpages. 

In [18]:
import pandas as pd
from bs4 import BeautifulSoup 
import os, requests, re, time, sys, inspect

In [8]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

#### File Locations

In [13]:
# Update with your own file structure
base_path = r"C:\Users\laure\Dropbox\!research\20181026_ihe_diversity"
# Path to the Brooking's data with the names of the universities
brookings_path = os.path.join(base_path,'data','brookings_value_added')
sm_path = os.path.join(base_path,'data','social_media_links')
sm_py_path = os.path.join(base_path,'python_scraping','scrape_handles')

#### Import functions from .py
The functions in this file are used in the next step of this project as well. Because of this, I added the functions to ihe_scrape.py file, which I import below. To make it easier to see how these functions work, I use the inspect package to show the code for the functions in this notebook. 

In [43]:
social_media_py = str(os.path.join(sm_py_path, 'social_media_urls.py'))
%run $social_media_py

In [11]:
def print_func(func):
    lines = inspect.getsource(func)
    print(lines)

## Base Data on Colleges to Search

The base IHE data comes from the Brookings college value-added rankings.

In [4]:
b_df = pd.read_csv(os.path.join(brookings_path,'brookings_4yr_value_added.csv'))
b_df.head(2)

,unitid,instnm,ccbasic,pct_baplus_awards,primarily_four_year,city,omb_cbsa,omb_title,stabbr,omb_state_name,...,BSALg_z_pct_STEM_award_HIGH_ANY,BSALg_z_lnMed_inc_field,BSALg_z_grad_rate200,BOCC_z_Retention_Rate2009,BOCC_z_avg_inst_aid,BOCC_z_lnavg_inst_sal,BOCC_z_lnsch_skill_val,BOCC_z_pct_STEM_award_HIGH_ANY,BOCC_z_lnMed_inc_field,BOCC_z_grad_rate200
0,372213,University of Phoenix-Online Campus,Doctoral/Research Universities,64%,1,Phoenix,38060.0,"Phoenix-Mesa-Scottsdale, AZ",AZ,Arizona,...,0.3%,4.8%,-2.6%,-0.6%,0.1%,-1.1%,2.9%,0.2%,0.3%,-0.1%
1,154022,Ashford University,Master's Colleges and Universities (larger pro...,90%,1,Clinton,17540.0,"Clinton, IA",IA,Iowa,...,-1.6%,1.6%,-1.8%,-0.4%,-0.2%,0.2%,-0.7%,-0.9%,0.1%,-0.1%


Keep only institutions with non-missing Value-Added data

In [5]:
b_df = b_df.dropna(subset=['VA_sal', 'VA_repay', 'VA_occ', 'VA_repay_broad', 'VA_sal_GRAD'])

Create list of schools concatenated with state. These will be implemented in Google search.

In [6]:
ihe_search = b_df.instnm + " " + b_df.omb_state_name
b_df = b_df.assign(ihe_search=ihe_search)

## Scrape Twitter URLS from Webpages

In [41]:
print_func(get_handles)

def get_handles(url, driver):
    
    # Get all text on page using Selenium
    driver.get(url)
    # Wait for page load
    for i in range(120):
        page_state = driver.execute_script('return document.readyState;')
        try:
            if page_state=='complete': break
        except: pass
        time.sleep(0.5)
    else: self.fail("time out")    
    text = driver.page_source
    
    # Get all links on page that contain Twitter or Facebook using BeautifulSoup
    root = BeautifulSoup(text, "lxml")
    
    # Twitter
    twitter_links = []
    for link in root.find_all("a", href=re.compile("twitter")):
        twitter_links.append(link.get('href'))
    
    # Remove individual twitter post links by removing links from list that have two consecutive numbers
    twitter_links = [link for link in twitter_links if bool(re.search(r'\d\d', link)) == False]
    
    # Facebook
    facebook_links = []
    for link in root.find_all("a", href=re.compile("facebook")):
        facebook_

In [44]:
# Test
# Switch to 1==1 to turn on
if 1==2:
    options = Options()
    options.headless = True
    driver = webdriver.Firefox(options=options)
    url = r"https://www.ucf.edu/"
    twitter_links, facebook_links, instagram_links = get_handles(url=url,driver=driver)
    print(str(twitter_links) + str(facebook_links) + str(instagram_links))
    driver.quit()

['https://www.twitter.com/ucf', 'https://twitter.com/UCF']['https://www.facebook.com/ucf', 'https://www.facebook.com/UCF']['https://www.instagram.com/ucf.edu/', 'https://www.instagram.com/ucf.edu/']


## Use "I'm Feeling Lucky" to Search for Pages and Get URLS

In [45]:
print_func(feeling_lucky)

def feeling_lucky(search_target, driver):
    
    
    driver.get("http://www.google.com/")
    time.sleep(1)
    
    # Input university to search for
    driver.find_element_by_name("q").send_keys(search_target)
    time.sleep(1)
    driver.find_element_by_name("q").send_keys(Keys.TAB)
    time.sleep(1)
    
    # Click on get lucky button 
    driver.find_element_by_id("gbqfbb").click()
    wait = WebDriverWait(driver, 20)
    wait.until(lambda driver: 'google' not in driver.current_url)
    
    page_url = driver.current_url
    
    return page_url 



In [49]:
# Test
if 1==1:
    options = Options()
    options.headless = True
    driver = webdriver.Firefox(options=options)
    print(feeling_lucky("University of Washington", driver=driver))
    driver.quit()

https://www.washington.edu/


## Remove Duplicate URLs

In [51]:
print_func(rm_dup)

def rm_dup(a_list): 
    final_list = [] 
    for item in a_list: 
        if item not in final_list: 
            final_list.append(item) 
    return final_list 



In [52]:
if 1==1:
    test_list = ['a', 'a', 'b']
    print(rm_dup(test_list))

['a', 'b']


## Put it togther: Find Twitter, Facebook, and Instagram URLs for Colleges

In [15]:
# Write csv header
if not os.path.isfile(os.path.join(sm_path, "ihe_sm_pages_v2.csv")):
    with open(os.path.join(sm_path, "ihe_sm_pages_v2.csv"), 'w') as f:
        print('unitid; instnm; url_main; url_admissions; twitter_links; facebook_links; instagram_links', file=f)

In [16]:
# Initialize headless Firefox browser
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

for index, row in b_df.iterrows():
    try:
        # Get URL from base webpage
        ihe_url = feeling_lucky(row['ihe_search'], driver=driver)

        # Get Twitter links
        main_twitter_links, main_facebook_links, main_instagram_links = get_handles(url=ihe_url, driver=driver)

        # Also search for admissions page to see if Twitter is different
        admissions_url = feeling_lucky(row['ihe_search'] + ' admissions', driver=driver)

        # Get Twitter links for admissions page
        adm_twitter_links, adm_facebook_links, adm_instagram_links = get_handles(url=admissions_url, driver=driver)

        # If no links found, add NA
        # Twitter
        if not main_twitter_links and not adm_twitter_links:
            main_twitter_links = ['No Twitter found']
        if not main_facebook_links and not adm_facebook_links:
            main_facebook_links = ['No Facebook found']
        if not main_instagram_links and not adm_instagram_links:
            main_instagram_links = ['No Instagram found']

        # Remove any duplicate links
        twitter_links = rm_dup(main_twitter_links + adm_twitter_links)
        facebook_links = rm_dup(main_facebook_links + adm_facebook_links)
        instagram_links = rm_dup(main_instagram_links + adm_instagram_links)

        # Pring on page and write to csv file
        print(str(row['unitid']) + ',' + row['instnm'] + ',' + ihe_url + ',' 
                  + admissions_url + ',' + str(twitter_links[0]) + ',' 
                  + str(facebook_links[0]) + ',' + str(instagram_links[0]))
        with open(os.path.join(sm_path, "ihe_sm_pages_v2.csv"), 'a') as f:
            print(str(row['unitid']) + ';' + row['instnm'] + ';' + ihe_url + ';' 
                + admissions_url + ';' + str(twitter_links) + ';' 
                + str(facebook_links) + ';' + str(instagram_links), file=f)
    
    except:
        print("Error: " + str(row['unitid']) + ' - ' + row['instnm'])
        with open(os.path.join(sm_path, "ihe_sm_pages_v2.csv"), 'a') as f:
            print(str(row['unitid']) + ';' + row['instnm'] + ';' + "Error" + ';' + "Error" + ';' + "Error" + ';' + "Error"+ ';' + "Error", file=f)
            
driver.quit()
    

154022,Ashford University,https://www.ashford.edu/,https://www.ashford.edu/online-admissions,https://twitter.com/AshfordU,https://www.facebook.com/ashforduniversity,https://instagram.com/ashfordu
132903,University of Central Florida,https://www.ucf.edu/,https://www.ucf.edu/admissions/,https://www.twitter.com/ucf,https://www.facebook.com/ucf,https://www.instagram.com/ucf.edu/
134130,University of Florida,http://www.ufl.edu/,https://admissions.ufl.edu/,https://twitter.com/UF/,http://www.facebook.com/uflorida/,https://instagram.com/uflorida/
193900,New York University,https://www.nyu.edu/,https://www.nyu.edu/admissions/undergraduate-admissions.html,https://twitter.com/nyuniversity,https://www.facebook.com/NYU,https://www.instagram.com/p/Bkfnfy1F0Mg/
232557,Liberty University,http://www.liberty.edu/,https://www.liberty.edu/admissions/,http://twitter.com/libertyu?utm_source=liberty.edu&utm_medium=footer&utm_campaign=lu_twitter,http://www.facebook.com/LibertyUniversity?utm_source=liberty.edu

110608,California State University-Northridge,https://www.csun.edu/,https://www.csun.edu/admissions-records,https://twitter.com/csunorthridge,https://www.facebook.com/calstatenorthridge,https://www.instagram.com/csun_edu/
232186,George Mason University,https://www2.gmu.edu/,https://www2.gmu.edu/admissions-aid,https://twitter.com/georgemasonu,http://www.facebook.com/georgemason,https://instagram.com/georgemasonu
227216,University of North Texas,https://www.unt.edu/,https://admissions.unt.edu/,https://twitter.com/UNTsocial,https://www.facebook.com/northtexas,https://www.instagram.com/UNT/
201885,University of Cincinnati-Main Campus,https://www.uc.edu/,https://admissions.uc.edu/,http://twitter.com/UofCincy,http://www.facebook.com/uofcincinnati,http://instagram.com/uofcincy
216339,Temple University,https://www.temple.edu/,https://www.temple.edu/admissions,https://twitter.com/templeuniv,https://www.facebook.com/templeu,https://instagram.com/templeuniv
225511,University of Houston,http://www

151111,Indiana University-Purdue University-Indianapolis,https://www.iupui.edu/,https://www.iupui.edu/,https://twitter.com/iupui,https://www.facebook.com/IUPUI/,https://www.instagram.com/p/BpZv1N_AW32/
144740,DePaul University,https://www.depaul.edu/Pages/default.aspx,https://www.depaul.edu/admission-and-aid/Pages/default.aspx,https://twitter.com/depaulu,https://www.facebook.com/DePaulUniversity/,https://www.instagram.com/depaulu/
196097,Stony Brook University,https://www.stonybrook.edu/,https://www.stonybrook.edu/undergraduate-admissions/,https://twitter.com/stonybrooku,https://www.facebook.com/stonybrooku,http://instagram.com/stonybrooku
450933,Columbia Southern University,https://www.columbiasouthern.edu/,https://www.columbiasouthern.edu/admissions,No Twitter found,https://www.facebook.com/Columbia.Southern,http://instagram.com/columbiasouthernuniversity
155317,University of Kansas,https://ku.edu/,https://admissions.ku.edu/,https://twitter.com/KUnews,https://www.facebook.com/KU/,htt

181464,University of Nebraska-Lincoln,https://www.unl.edu/,https://admissions.unl.edu/,https://twitter.com/UNLincoln,https://www.facebook.com/sharer/sharer.php?u=https%3A%2F%2Fwww.unl.edu%2F%3Futm_campaign%3Dwdn_social%26utm_medium%3Dshare_this%26utm_source%3Dwdn_facebook,https://www.instagram.com/p/BpQRkTaBES9/
149222,Southern Illinois University-Carbondale,https://siu.edu/,https://admissions.siu.edu/,https://twitter.com/SIUCAdmissions,https://www.facebook.com/SIUC.Admissions/,https://www.instagram.com/siucadmissions/
198419,Duke University,https://www.duke.edu/,https://admissions.duke.edu/,https://twitter.com/DukeU,https://www.facebook.com/DukeUniv,http://instagram.com/dukeuniversity
182281,University of Nevada-Las Vegas,https://www.unlv.edu/,https://www.unlv.edu/admissions/freshman,https://twitter.com/UNLV,https://www.facebook.com/OfficialUNLV,https://www.instagram.com/unlv
110671,University of California-Riverside,https://www.ucr.edu/,http://admissions.ucr.edu/,https://twitter.com/

185590,Montclair State University,https://www.montclair.edu/,https://www.montclair.edu/university-admissions/,https://twitter.com/montclairstateu,https://www.facebook.com/MontclairStateUniversity,No Instagram found
197869,Appalachian State University,https://www.appstate.edu/,https://www.appstate.edu/admissions/,https://twitter.com/appstate,https://www.facebook.com/appalachianstateuniversity,https://instagram.com/appstate/
221999,Vanderbilt University,https://www.vanderbilt.edu/,https://admissions.vanderbilt.edu/,http://twitter.com/vanderbiltu,http://facebook.com/vanderbilt,http://instagram.com/vanderbiltu
220862,University of Memphis,https://www.memphis.edu/,https://www.memphis.edu/admissions/,https://twitter.com/uofmemphis,https://www.facebook.com/uofmemphis,https://www.instagram.com/uofmemphis/
211440,Carnegie Mellon University,https://www.cmu.edu/,https://admission.enrollment.cmu.edu/,http://www.twitter.com/carnegiemellon,http://www.facebook.com/carnegiemellonu,https://instagram.co

...

240374,University of Wisconsin-Parkside,https://www.uwp.edu/,https://www.uwp.edu/apply/admissions/,https://twitter.com/uwparkside,https://www.facebook.com/universityofwisconsinparkside,https://www.instagram.com/uw_parkside_admissions/
183974,Centenary College,http://www.centenaryuniversity.edu/,http://www.centenaryuniversity.edu/freshmen/,https://twitter.com/Centenary_NJ,https://www.facebook.com/centenaryuniversity,https://www.instagram.com/centenaryuniversity
Error: 204635 - Ohio Northern University
214175,Muhlenberg College,https://www.muhlenberg.edu/,https://www.muhlenberg.edu/main/admissions/,http://twitter.com/muhlenberg,http://www.facebook.com/MuhlenbergCollege,https://www.instagram.com/p/BpZzITfnmHZ/
126669,Colorado Christian University,https://www.ccu.edu/,https://www.ccu.edu/admissions/,https://twitter.com/my_ccu,https://www.facebook.com/myCCU,https://instagram.com/myccu
166939,Mount Holyoke College,https://www.mtholyoke.edu/,https://www.mtholyoke.edu/admission,https://twitter

202523,Denison University,https://denison.edu/,https://denison.edu/campus/admission,https://twitter.com/denisonu,https://www.facebook.com/denisonuniversity,https://www.instagram.com/denisonu
209506,Oregon Institute of Technology,https://www.oit.edu/,https://www.oit.edu/admissions,https://twitter.com/OregonTech,https://www.facebook.com/OregonTech,https://www.instagram.com/oregontech/
184694,Fairleigh Dickinson University-College at Florham,https://www.fdu.edu/,https://view2.fdu.edu/admissions/,https://twitter.com/FDUWhatsNew,https://www.facebook.com/fairleighdickinsonuniversity,https://instagram.com/fduwhatsnew#
155520,MidAmerica Nazarene University,https://www.mnu.edu/,https://www.mnu.edu/undergraduate/admissions,http://twitter.com/followMNU,https://www.facebook.com/MNUPioneers/,https://www.instagram.com/mnupioneers/
184773,Georgian Court University,https://georgian.edu/,https://georgian.edu/admissions/,https://twitter.com/Georgiancourt,https://www.facebook.com/georgiancourtu/,https://

163295,Maryland Institute College of Art,https://www.mica.edu/,https://www.mica.edu/applying-to-mica/,http://www.twitter.com/mica,http://www.facebook.com/mica.edu,https://www.instagram.com/marylandinstitutecollegeofart/
133492,Eckerd College,https://www.eckerd.edu/,https://www.eckerd.edu/admissions/,https://www.twitter.com/eckerdcollege,https://www.facebook.com/eckerdcollege,https://www.instagram.com/eckerdlife
239318,Milwaukee School of Engineering,https://www.msoe.edu/,https://www.msoe.edu/admissions-aid/,https://twitter.com/MSOE,http://www.facebook.com/milwaukeeschoolofengineering,https://www.instagram.com/msoephoto/
110404,California Institute of Technology,http://www.caltech.edu/,http://www.caltech.edu/content/admissions,http://twitter.com/caltech,http://www.facebook.com/californiainstituteoftechnology,https://instagram.com/caltechedu
134079,Florida Southern College,https://www.flsouthern.edu/,https://www.flsouthern.edu/undergraduate/admissions.aspx,https://twitter.com/flsouthern/

110370,California College of the Arts,https://www.cca.edu/,https://www.cca.edu/admissions,http://www.twitter.com/CACollegeofArts,http://www.facebook.com/CaliforniaCollegeoftheArts,http://instagram.com/cacollegeofarts
212197,Elizabethtown College,https://www.etown.edu/,https://www.etown.edu/admissions/,http://twitter.com/etowncollege,http://www.facebook.com/etowncollege,http://instagram.com/etowncollege
216524,Ursinus College,https://www.ursinus.edu/,https://www.ursinus.edu/admission/,http://twitter.com/UrsinusAdmit,No Facebook found,No Instagram found
198385,Davidson College,https://www.davidson.edu/,https://www.davidson.edu/admission-and-financial-aid,https://www.twitter.com/DavidsonCollege,https://facebook.com/davidsoncollege.nc,https://instagram.com/davidsoncollege
218229,Lander University,https://www.lander.edu/,https://www.lander.edu/admissions,http://twitter.com/follow_lander,http://www.facebook.com/followlander,https://instagram.com/landeruniversity/
214157,Moravian College,http

120865,Pacific Union College,https://www.puc.edu/,https://www.puc.edu/admissions,http://twitter.com/pucnow,https://www.facebook.com/pacificunioncollege,https://www.instagram.com/pucnow
209922,Reed College,https://www.reed.edu/,https://www.reed.edu/apply/,https://twitter.com/Reed_College_,https://www.facebook.com/reedcollege,https://instagram.com/reedcollege
197984,Belmont Abbey College,https://belmontabbeycollege.edu/,https://belmontabbeycollege.edu/admissions/,https://twitter.com/BelmontAbbey,https://www.facebook.com/belmontabbey/,https://www.instagram.com/belmontabbey/?hl=en
100937,Birmingham Southern College,https://www.bsc.edu/,https://www.bsc.edu/admission/,https://twitter.com/FromTheHilltop,https://www.facebook.com/43199567384/posts/10155533542147385,https://www.instagram.com/p/BmwOlGPnwS3/
101435,Huntingdon College,https://www.huntingdon.edu/,https://www.collegesimply.com/colleges/alabama/huntingdon-college/admission/,https://twitter.com/HuntingdonColl,https://www.facebook.com/H